In [75]:
%pip install google-api-python-client
%pip install tk


Note: you may need to restart the kernel to use updated packages.


In [11]:
import googleapiclient.discovery
from tqdm import tqdm
from time import sleep
import tkinter as tk
import keys

api_service_name = "youtube"
api_version = "v3"

key = keys.api_key
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=key)

def getLinks(timeStr, maxCount):
    time = timeStr.split(':')
    minutes = str(int(time[0]))
    seconds = str(int(time[1]))
    duration = 'short'
    if int(minutes) >= 4:
        duration = 'medium'
    if int(minutes) > 20:
        duration = 'long'
    print(duration)
    outIds=[]
    count = 0
    with tqdm(total=maxCount) as pbar:
        while count < maxCount:
            print('outer looping')
            listRequest = youtube.search().list(
                    part='snippet',
                    type='video',
                    videoDuration=duration,
                    q=minutes + ' minute'
                )
            listResponse = listRequest.execute()
            while 'nextPageToken' in listResponse.keys() and count < maxCount:
                nextPageToken = listResponse['nextPageToken']
                listRequest = youtube.search().list(
                    part='snippet',
                    type='video',
                    videoDuration=duration,
                    maxResults=50,
                    pageToken=nextPageToken,
                    q=minutes + ' minute'
                )
                listResponse = listRequest.execute()
                videos = listResponse['items']
                if videos is None or len(videos) == 0:
                    print("empty videos, breaking")
                    break
                ids = []
                for video in videos:
                    ids.append(video['id']['videoId'])
                request = youtube.videos().list(
                    part='contentDetails',
                    id=ids
                )
                response = request.execute()
                items = response['items']
                for item in items:
                    sleep(0.01)
                    count+=1
                    pbar.update(1)
                    isoTime = item['contentDetails']['duration']
                    timeArr = isoTime.split('M')
                    tempMin = ''.join(c for c in timeArr[0] if c.isdigit()) if len(timeArr) > 1 else "0"
                    tempSec = ''.join(c for c in timeArr[1] if c.isdigit()) if len(timeArr) > 1 else ''.join(c for c in timeArr[0] if c.isdigit())
                    if minutes == tempMin and seconds == tempSec:
                        print('found one!')
                        outIds.append(item['id'])
    outLinks = []
    for outId in outIds:
        outLinks.append('https://www.youtube.com/watch?v='+outId)
    return outLinks

def getNumCount(s):
    return sum(c.isdigit() for c in s)

In [18]:
time = '9:28'
maxCount = 500

links = getLinks(time, maxCount)
links.sort(key=getNumCount)
print(links)

medium


  0%|          | 0/500 [00:00<?, ?it/s]

outer looping


 57%|█████▋    | 287/500 [00:09<00:04, 46.63it/s]

found one!


100%|██████████| 500/500 [00:17<00:00, 29.14it/s]


['https://www.youtube.com/watch?v=iY7BrTIXVkk']
